<a href="https://colab.research.google.com/github/SrivardhanS/Speech_GNN_FYP/blob/main/fyp_oct_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ======================
# CELL 1: Dataset Download & Preparation
# Run this cell ONCE to download and prepare the dataset
# ======================

# Install dependencies
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-2.4.0+cpu.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-2.4.0+cpu.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-2.4.0+cpu.html
!pip install -q torch-spline-conv -f https://data.pyg.org/whl/torch-2.4.0+cpu.html
!pip install -q torch-geometric
!pip install -q kaggle librosa

import os, sys, logging
import pandas as pd
import numpy as np
import kagglehub

# Logging setup
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
    force=True
)
logger = logging.getLogger("asvspoof-dataset")

# Download dataset
path = kagglehub.dataset_download("awsaf49/asvpoof-2019-dataset")
logger.info(f"Dataset path: {path}")

# Set up paths
dataset_path = os.path.join(path, "LA", "LA")
proto_dir = os.path.join(dataset_path, "ASVspoof2019_LA_cm_protocols")
train_proto = os.path.join(proto_dir, "ASVspoof2019.LA.cm.train.trn.txt")
dev_proto = os.path.join(proto_dir, "ASVspoof2019.LA.cm.dev.trl.txt")
eval_proto = os.path.join(proto_dir, "ASVspoof2019.LA.cm.eval.trl.txt")

train_audio_dir = os.path.join(dataset_path, "ASVspoof2019_LA_train", "flac")
dev_audio_dir   = os.path.join(dataset_path, "ASVspoof2019_LA_dev", "flac")
eval_audio_dir  = os.path.join(dataset_path, "ASVspoof2019_LA_eval", "flac")

# Load protocols
protocol_df = pd.read_csv(train_proto, sep=" ", header=None)
protocol_df.columns = ["utt_id", "speaker_id", "system_id", "attack_id", "label"]
logger.info(f"Train protocol sample:\n{protocol_df.head()}")
logger.info(f"Train label distribution:\n{protocol_df['label'].value_counts()}")

dev_df = pd.read_csv(dev_proto, sep=" ", header=None)
dev_df.columns = ["utt_id", "speaker_id", "system_id", "attack_id", "label"]
logger.info(f"Dev protocol loaded: {len(dev_df)} samples")
logger.info(f"Dev label distribution:\n{dev_df['label'].value_counts()}")

eval_df = pd.read_csv(eval_proto, sep=" ", header=None)
eval_df.columns = ["utt_id", "speaker_id", "system_id", "attack_id", "label"]
logger.info(f"Eval protocol loaded: {len(eval_df)} samples")
logger.info(f"Eval label distribution:\n{eval_df['label'].value_counts()}")

logger.info("✅ Dataset preparation complete! You can now run Cell 2 for training.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.9/542.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.6/792.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.5 MB/s eta 0:00:00
Using Colab cache for faster access to the 'asvpoof-2019-dataset' dataset.
13:07:15 [INFO] Dataset path: /kaggle/input/asvpoof-2019-dataset
13:07:16 [INFO] Train protocol sample:
    utt_id    speaker_id system_id attack_id     label
0  LA_0079  LA_T_1138215         -         -  bonafide
1  LA_0079  LA_T_1271820         -         -  bonafide
2  LA_0079  LA_T_1272637         -         -  bonafide
3  LA_0079  LA_T_1276960         -         -  bonafide
4  LA_0079  LA_T_1341447         -         -  bonafide
13:07:16 [IN

In [ ]:
# ======================
# CELL 2: Model Training & Evaluation - FULLY BALANCED
# Uses ALL bonafide samples and matches with equal number of spoof samples
# Train: 2580 bonafide + 2580 spoof = 5160 total
# Dev: 2548 bonafide + 2548 spoof = 5096 total
# Eval: 7355 bonafide + 7355 spoof = 14710 total
# ======================

import os, sys, logging, random
import pandas as pd
import numpy as np
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, confusion_matrix,
                             classification_report)

# Logging setup
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
    force=True
)
logger = logging.getLogger("asvspoof-train")

# ----------------------
# DATASET STATISTICS
# ----------------------
logger.info("\n" + "="*60)
logger.info("BALANCED DATASET STRATEGY")
logger.info("="*60)
logger.info(f"Train: Use ALL {len(protocol_df[protocol_df['label']=='bonafide'])} bonafide + {len(protocol_df[protocol_df['label']=='bonafide'])} spoof")
logger.info(f"Dev: Use ALL {len(dev_df[dev_df['label']=='bonafide'])} bonafide + {len(dev_df[dev_df['label']=='bonafide'])} spoof")
logger.info(f"Eval: Use ALL {len(eval_df[eval_df['label']=='bonafide'])} bonafide + {len(eval_df[eval_df['label']=='bonafide'])} spoof")
logger.info("="*60 + "\n")

# ----------------------
# audio -> temporal graph
# ----------------------
def audio_to_graph(file_path, sr=16000, n_mfcc=13, pool_size=4):
    y, _ = librosa.load(file_path, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)

    T = mfcc.shape[1] // pool_size
    if T < 2:
        mfcc = np.pad(mfcc, ((0,0),(0, pool_size*2 - mfcc.shape[1])), mode='wrap')
        T = mfcc.shape[1] // pool_size
    pooled = np.stack([ np.mean(mfcc[:, i*pool_size:(i+1)*pool_size], axis=1) for i in range(T) ])

    x = torch.tensor(pooled, dtype=torch.float)
    if x.size(0) < 2:
        x = torch.cat([x, x], dim=0)
    edge_index = torch.tensor([[i,i+1] for i in range(x.size(0)-1)], dtype=torch.long).T
    return Data(x=x, edge_index=edge_index)

# ----------------------
# DifferentiableGraphCompressor
# ----------------------
class DifferentiableGraphCompressor(nn.Module):
    def __init__(self, feature_dim, tau_T=1.0, lambda_id=1.0, lambda_comp=0.1):
        super().__init__()
        self.a_raw = nn.Parameter(torch.tensor(0.0))
        self.tau_T = tau_T
        self.lambda_id = lambda_id
        self.lambda_comp = lambda_comp
        self.speaker_embedding = nn.Sequential(
            nn.Linear(feature_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )

    def get_alpha(self):
        return torch.sigmoid(self.a_raw)

    def node_similarity(self, x_u, x_v):
        return F.cosine_similarity(x_u.unsqueeze(0), x_v.unsqueeze(0), dim=1)

    def neighborhood_similarity(self, x, edge_index, u, v):
        def get_neighbors(node):
            mask = (edge_index[0] == node)
            if mask.any():
                return edge_index[1][mask]
            else:
                return torch.tensor([], dtype=torch.long, device=x.device)

        neighbors_u = get_neighbors(u)
        neighbors_v = get_neighbors(v)
        if neighbors_u.numel() == 0 or neighbors_v.numel() == 0:
            return torch.tensor(0.0, device=x.device)
        m_u = x[neighbors_u].mean(dim=0)
        m_v = x[neighbors_v].mean(dim=0)
        return F.cosine_similarity(m_u.unsqueeze(0), m_v.unsqueeze(0), dim=1)

    def combined_similarity(self, x, edge_index, u, v):
        alpha = self.get_alpha()
        sim_x = self.node_similarity(x[u], x[v])
        sim_m = self.neighborhood_similarity(x, edge_index, u, v)
        return alpha * sim_x + (1 - alpha) * sim_m

    def compute_adaptive_thresholds(self, similarities):
        tau_1_bar = torch.quantile(similarities, 0.75)
        tau_2_bar = torch.quantile(similarities, 0.60)
        return tau_1_bar, tau_2_bar

    def compute_merge_probabilities(self, x, edge_index, window_size=10):
        num_nodes = x.size(0)
        merge_probs = torch.zeros(num_nodes, device=x.device)
        all_similarities = []

        for t in range(1, num_nodes - 1):
            for k in range(1, min(window_size + 1, num_nodes - t)):
                if t + k >= num_nodes - 1:
                    continue
                try:
                    s1 = self.combined_similarity(x, edge_index, t, t + k)
                    s2 = self.combined_similarity(x, edge_index, t - 1, t + k - 1)
                    s3 = self.combined_similarity(x, edge_index, t + 1, t + k + 1)
                    all_similarities.extend([s1, s2, s3])
                except Exception:
                    continue

        if len(all_similarities) == 0:
            return merge_probs

        similarities_tensor = torch.stack(all_similarities).view(-1)
        tau_1_bar, tau_2_bar = self.compute_adaptive_thresholds(similarities_tensor)

        for t in range(1, num_nodes - 1):
            max_prob = 0.0
            for k in range(1, min(window_size + 1, num_nodes - t)):
                if t + k >= num_nodes - 1:
                    continue
                try:
                    s1 = self.combined_similarity(x, edge_index, t, t + k)
                    s2 = self.combined_similarity(x, edge_index, t - 1, t + k - 1)
                    s3 = self.combined_similarity(x, edge_index, t + 1, t + k + 1)
                    gate1 = torch.sigmoid((s1 - tau_1_bar) / self.tau_T)
                    gate2 = torch.sigmoid((s2 - tau_2_bar) / self.tau_T)
                    gate3 = torch.sigmoid((s3 - tau_2_bar) / self.tau_T)
                    prob = gate1 * gate2 * gate3
                    max_prob = max(max_prob, prob.item())
                except Exception:
                    continue
            merge_probs[t] = max_prob

        return merge_probs

    def differentiable_compression(self, x, edge_index):
        merge_probs = self.compute_merge_probabilities(x, edge_index)
        x_compressed = x.clone()
        num_nodes = x.size(0)
        for t in range(1, num_nodes - 1):
            if merge_probs[t] > 0:
                best_k = 1
                best_sim = -1
                for k in range(1, min(10, num_nodes - t)):
                    if t + k >= num_nodes: break
                    try:
                        sim = self.combined_similarity(x, edge_index, t, t + k)
                        if sim > best_sim:
                            best_sim = sim
                            best_k = k
                    except Exception:
                        continue
                if t + best_k < num_nodes:
                    p = merge_probs[t]
                    interpolated = (x[t] + x[t + best_k]) / 2
                    x_compressed[t] = (1 - p) * x[t] + p * interpolated
        return x_compressed, merge_probs

    def speaker_identity_loss(self, x_original, x_compressed):
        g_original = self.speaker_embedding(x_original.mean(dim=0))
        g_compressed = self.speaker_embedding(x_compressed.mean(dim=0))
        cos_sim = F.cosine_similarity(g_original.unsqueeze(0), g_compressed.unsqueeze(0))
        return 1 - cos_sim

    def compression_loss(self, merge_probs):
        return (1 - merge_probs).mean()

    def forward(self, x, edge_index):
        x_compressed, merge_probs = self.differentiable_compression(x, edge_index)
        L_id = self.speaker_identity_loss(x, x_compressed)
        L_comp = self.compression_loss(merge_probs)
        total_loss = self.lambda_id * L_id + self.lambda_comp * L_comp
        return {
            'compressed_features': x_compressed,
            'merge_probs': merge_probs,
            'loss': total_loss,
            'loss_id': L_id,
            'loss_comp': L_comp,
            'alpha': self.get_alpha()
        }

    def hard_compression_inference(self, x, edge_index, window_size=10):
        self.eval()
        with torch.no_grad():
            alpha = self.get_alpha().item()
            num_nodes = x.size(0)
            to_remove = set()
            all_similarities = []
            for t in range(1, num_nodes - 1):
                for k in range(1, min(window_size + 1, num_nodes - t)):
                    if t + k >= num_nodes - 1: continue
                    try:
                        s1 = self.combined_similarity(x, edge_index, t, t + k)
                        s2 = self.combined_similarity(x, edge_index, t - 1, t + k - 1)
                        s3 = self.combined_similarity(x, edge_index, t + 1, t + k + 1)
                        all_similarities.extend([s1, s2, s3])
                    except Exception:
                        continue
            if all_similarities:
                similarities_tensor = torch.stack(all_similarities).view(-1)
                tau_1_hat, tau_2_hat = self.compute_adaptive_thresholds(similarities_tensor)
                for t in range(1, num_nodes - 1):
                    if t in to_remove: continue
                    for k in range(1, min(window_size + 1, num_nodes - t)):
                        if t + k >= num_nodes - 1 or any(n in to_remove for n in [t, t+k, t-1, t+k-1, t+1, t+k+1]):
                            continue
                        try:
                            s1 = self.combined_similarity(x, edge_index, t, t + k)
                            s2 = self.combined_similarity(x, edge_index, t - 1, t + k - 1)
                            s3 = self.combined_similarity(x, edge_index, t + 1, t + k + 1)
                            if s1 >= tau_1_hat and s2 >= tau_2_hat and s3 >= tau_2_hat:
                                to_remove.add(t)
                                break
                        except Exception:
                            continue
            remaining_indices = [i for i in range(num_nodes) if i not in to_remove]
            x_new = x[remaining_indices]
            index_mapping = {old_idx: new_idx for new_idx, old_idx in enumerate(remaining_indices)}
            new_edges = []
            for i in range(edge_index.size(1)):
                src, dst = edge_index[0, i].item(), edge_index[1, i].item()
                if src not in to_remove and dst not in to_remove:
                    new_edges.append([index_mapping[src], index_mapping[dst]])
            if new_edges:
                edge_index_new = torch.tensor(new_edges, dtype=torch.long).T
            else:
                edge_index_new = torch.empty((2, 0), dtype=torch.long)
            compressed_data = Data(x=x_new, edge_index=edge_index_new)
            return compressed_data, to_remove, alpha

# ----------------------
# GCN classifier
# ----------------------
class GCNClassifier(nn.Module):
    def __init__(self, in_channels, hidden_channels=32):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fc = nn.Linear(hidden_channels, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return torch.sigmoid(self.fc(x)).view(-1)

# ----------------------
# Build graphs - FULLY BALANCED (use ALL bonafide)
# ----------------------
def build_fully_balanced_graphs(df, audio_dir, n_mfcc=13, pool_size=4):
    """
    Uses ALL bonafide samples and randomly samples equal number of spoof samples
    """
    bonafide_df = df[df['label'] == 'bonafide']
    spoof_df = df[df['label'] == 'spoof']

    # Use ALL bonafide
    num_bonafide = len(bonafide_df)

    # Sample equal number of spoof
    spoof_sample = spoof_df.sample(n=num_bonafide, random_state=42)

    # Combine and shuffle
    balanced_df = pd.concat([bonafide_df, spoof_sample]).sample(frac=1, random_state=42).reset_index(drop=True)

    logger.info(f"Balanced dataset: {num_bonafide} bonafide + {len(spoof_sample)} spoof = {len(balanced_df)} total")

    graphs = []
    failed = 0
    for i, row in balanced_df.iterrows():
        utt = row['speaker_id']
        file_path = os.path.join(audio_dir, f"{utt}.flac")
        if not os.path.isfile(file_path):
            failed += 1
            continue
        try:
            g = audio_to_graph(file_path, n_mfcc=n_mfcc, pool_size=pool_size)
            g.y = torch.tensor(1 if row['label']=='bonafide' else 0, dtype=torch.float)
            graphs.append(g)
        except Exception as e:
            failed += 1
            continue

        if (i + 1) % 500 == 0:
            logger.info(f"  Processed {i+1}/{len(balanced_df)} samples...")

    labels = [int(g.y.item()) for g in graphs]
    logger.info(f"Final graphs: bonafide={labels.count(1)}, spoof={labels.count(0)} (failed={failed})")
    return graphs

# ----------------------
# Training + evaluation
# ----------------------
def train_epoch(compressor, classifier, loader, optimizer, criterion, device, epoch, lambda_comp_loss):
    classifier.train()
    compressor.train()
    total_cls_loss = 0.0
    total_comp_loss = 0.0
    n = 0
    for batch_idx, batch in enumerate(loader):
        batch = batch.to(device)
        x = batch.x
        edge_index = batch.edge_index

        comp_res = compressor(x, edge_index)
        x_compressed = comp_res['compressed_features']
        comp_loss = comp_res['loss']

        compressed_data = Data(x=x_compressed, edge_index=edge_index, y=batch.y,
                              batch=torch.zeros(x_compressed.size(0), dtype=torch.long, device=device))
        compressed_data = compressed_data.to(device)

        pred = classifier(compressed_data)
        cls_loss = criterion(pred, batch.y)

        loss = cls_loss + lambda_comp_loss * comp_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_cls_loss += cls_loss.item()
        total_comp_loss += comp_loss.item()
        n += 1

        if (batch_idx + 1) % 500 == 0:
            logger.info(f"Epoch {epoch} | Batch {batch_idx+1}/{len(loader)} | cls_loss={cls_loss.item():.4f} comp_loss={comp_loss.item():.4f}")

    return total_cls_loss / n, total_comp_loss / n

def evaluate_with_hard_compression(compressor, classifier, loader, device):
    classifier.eval()
    compressor.eval()
    y_true, y_pred, y_prob = [], [], []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            compressed_data, removed, alpha = compressor.hard_compression_inference(batch.x, batch.edge_index)
            compressed_data.y = batch.y.cpu()
            compressed_data.batch = torch.zeros(compressed_data.x.size(0), dtype=torch.long)
            compressed_data = compressed_data.to(device)

            prob = classifier(compressed_data)
            pred = (prob >= 0.5).long()
            y_true.append(int(batch.y.cpu().item()))
            y_pred.append(int(pred.cpu().item()))
            y_prob.append(float(prob.cpu().item()))

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    auc = roc_auc_score(y_true, y_prob) if len(set(y_true)) > 1 else float('nan')
    return {"acc":acc, "prec":prec, "rec":rec, "f1":f1, "auc":auc,
            "y_true": y_true, "y_pred": y_pred, "y_prob": y_prob}


# ======================
# TRAINING STARTS HERE
# ======================
save_path = "/content/gcn_compressed_asvspoof.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

# Build FULLY BALANCED datasets
logger.info("\nBuilding FULLY BALANCED training graphs...")
train_graphs = build_fully_balanced_graphs(protocol_df, train_audio_dir)
train_loader = DataLoader(train_graphs, batch_size=1, shuffle=True)

logger.info("\nBuilding FULLY BALANCED dev graphs...")
dev_graphs = build_fully_balanced_graphs(dev_df, dev_audio_dir)
dev_loader = DataLoader(dev_graphs, batch_size=1, shuffle=False)

logger.info("\nBuilding FULLY BALANCED eval graphs...")
eval_graphs = build_fully_balanced_graphs(eval_df, eval_audio_dir)
eval_loader = DataLoader(eval_graphs, batch_size=1, shuffle=False)

# Initialize models
feature_dim = 13
compressor = DifferentiableGraphCompressor(feature_dim, tau_T=1.0, lambda_id=1.0, lambda_comp=0.05).to(device)
classifier = GCNClassifier(in_channels=feature_dim, hidden_channels=32).to(device)

# Training setup - BALANCED LOSS (no weighting needed since data is balanced)
criterion = nn.BCELoss()
optimizer = optim.Adam(
    list(compressor.parameters()) + list(classifier.parameters()), lr=1e-3
)
lambda_comp_loss = 0.1
epochs = 6

# Train with validation
logger.info("\n" + "="*60)
logger.info("STARTING TRAINING ON FULLY BALANCED DATASET")
logger.info("="*60)

best_dev_f1 = 0.0
for epoch in range(1, epochs + 1):
    cls_loss, comp_loss = train_epoch(
        compressor, classifier, train_loader, optimizer, criterion, device, epoch, lambda_comp_loss
    )
    logger.info(f"Epoch {epoch}: cls_loss={cls_loss:.4f} | comp_loss={comp_loss:.4f}")

    dev_metrics = evaluate_with_hard_compression(compressor, classifier, dev_loader, device)
    logger.info(f"Epoch {epoch} Dev: acc={dev_metrics['acc']:.4f}, f1={dev_metrics['f1']:.4f}, auc={dev_metrics['auc']}")

    if dev_metrics['f1'] > best_dev_f1:
        best_dev_f1 = dev_metrics['f1']
        torch.save({
            "compressor_state": compressor.state_dict(),
            "classifier_state": classifier.state_dict(),
            "epoch": epoch,
            "dev_metrics": dev_metrics
        }, save_path)
        logger.info(f"✅ Best model saved (Dev F1: {best_dev_f1:.4f})")

# ======================
# FINAL EVALUATION
# ======================
logger.info("\n" + "="*60)
logger.info("FINAL EVALUATION ON ALL SETS")
logger.info("="*60)

ckpt = torch.load(save_path, map_location=device, weights_only=False)
compressor.load_state_dict(ckpt['compressor_state'])
classifier.load_state_dict(ckpt['classifier_state'])
logger.info(f"Loaded best model from epoch {ckpt.get('epoch', '?')}")

# Evaluate on training set
logger.info("\n--- Training Set Results ---")
train_metrics = evaluate_with_hard_compression(compressor, classifier, train_loader, device)
logger.info(f"Accuracy : {train_metrics['acc']:.4f}")
logger.info(f"Precision: {train_metrics['prec']:.4f}")
logger.info(f"Recall   : {train_metrics['rec']:.4f}")
logger.info(f"F1-score : {train_metrics['f1']:.4f}")
logger.info(f"AUC      : {train_metrics['auc']}")
logger.info("\nConfusion Matrix:")
print(confusion_matrix(train_metrics['y_true'], train_metrics['y_pred']))
logger.info("Classification Report:")
print(classification_report(train_metrics['y_true'], train_metrics['y_pred'],
                           target_names=['spoof', 'bonafide'], digits=4, zero_division=0))

# Evaluate on dev set
logger.info("\n--- Dev Set Results ---")
dev_metrics = evaluate_with_hard_compression(compressor, classifier, dev_loader, device)
logger.info(f"Accuracy : {dev_metrics['acc']:.4f}")
logger.info(f"Precision: {dev_metrics['prec']:.4f}")
logger.info(f"Recall   : {dev_metrics['rec']:.4f}")
logger.info(f"F1-score : {dev_metrics['f1']:.4f}")
logger.info(f"AUC      : {dev_metrics['auc']}")
logger.info("\nConfusion Matrix:")
print(confusion_matrix(dev_metrics['y_true'], dev_metrics['y_pred']))
logger.info("Classification Report:")
print(classification_report(dev_metrics['y_true'], dev_metrics['y_pred'],
                           target_names=['spoof', 'bonafide'], digits=4, zero_division=0))

# Evaluate on test/eval set
logger.info("\n--- Test Set Results ---")
eval_metrics = evaluate_with_hard_compression(compressor, classifier, eval_loader, device)
logger.info(f"Accuracy : {eval_metrics['acc']:.4f}")
logger.info(f"Precision: {eval_metrics['prec']:.4f}")
logger.info(f"Recall   : {eval_metrics['rec']:.4f}")
logger.info(f"F1-score : {eval_metrics['f1']:.4f}")
logger.info(f"AUC      : {eval_metrics['auc']}")
logger.info("\nConfusion Matrix:")
print(confusion_matrix(eval_metrics['y_true'], eval_metrics['y_pred']))
logger.info("Classification Report:")
print(classification_report(eval_metrics['y_true'], eval_metrics['y_pred'],
                           target_names=['spoof', 'bonafide'], digits=4, zero_division=0))

logger.info("\n" + "="*60)
logger.info("EVALUATION COMPLETE")
logger.info("="*60)

/usr/local/lib/python3.12/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.12/dist-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.12/dist-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /usr/local/lib/python3.12/dist-packages/torch_cluster/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/usr/local/lib/python3.12/dist-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /usr/local/lib/python3.12/dist-packages/torch_spline_conv/_version_cpu.so: undefined symbol: _ZN

13:07:48 [INFO] 
13:07:48 [INFO] BALANCED DATASET STRATEGY
13:07:48 [INFO] ============================================================
13:07:48 [INFO] Train: Use ALL 2580 bonafide + 2580 spoof
13:07:48 [INFO] Dev: Use ALL 2548 bonafide + 2548 spoof
13:07:48 [INFO] Eval: Use ALL 7355 bonafide + 7355 spoof
13:07:48 [INFO] ============================================================

13:07:48 [INFO] Using device: cpu
13:07:48 [INFO] 
Building FULLY BALANCED training graphs...
13:07:48 [INFO] Balanced dataset: 2580 bonafide + 2580 spoof = 5160 total
13:08:20 [INFO]   Processed 500/5160 samples...
13:08:34 [INFO]   Processed 1000/5160 samples...
13:08:48 [INFO]   Processed 1500/5160 samples...
13:09:02 [INFO]   Processed 2000/5160 samples...
13:09:16 [INFO]   Processed 2500/5160 samples...
13:09:31 [INFO]   Processed 3000/5160 samples...
13:09:46 [INFO]   Processed 3500/5160 samples...
13:10:00 [INFO]   Processed 4000/5160 samples...
13:10:14 [INFO]   Processed 4500/5160 samples...
13:10:2